## Loading packages

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

## Downloading data

In [2]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "dailyrainfall/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [4]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 3.63 s, sys: 4.52 s, total: 8.15 s
Wall time: 1min 39s


### Extracting ZIP

In [5]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 14.5 s, sys: 1.91 s, total: 16.4 s
Wall time: 16.6 s


In [6]:
%ls -ltr dailyrainfall/

total 12428672
drwxr-xr-x  30 apple  staff        960 Mar 28 16:48 __MACOSX/
-rw-r--r--   1 apple  staff  814041183 Mar 28 16:51 data.zip
-rw-r--r--   1 apple  staff   95376895 Mar 28 16:51 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff   94960113 Mar 28 16:51 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff   82474546 Mar 28 16:51 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  127613760 Mar 28 16:51 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  232118894 Mar 28 16:51 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  330360682 Mar 28 16:51 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  254009247 Mar 28 16:51 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  235661418 Mar 28 16:51 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  294260911 Mar 28 16:51 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  295768615 Mar 28 16:51 EC-Earth3-Veg-LR_daily_ra